In [121]:
import os
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import urllib
import datetime
import time

active_month = "Mar"

In [2]:
time.strftime("%m%d%Y")

'05232016'

In [3]:
start = time.time()
print "hello Jalal Jahir what"
print "hello Jalal Jahir is a GREAT Guy"
end = time.time()
processtime = end - start
print "{0: .3}".format(processtime)
print float(processtime)

hello Jalal Jahir what
hello Jalal Jahir is a GREAT Guy
 0.0
0.0


In [4]:
print os.getcwd()

J:\Staff\Jalal\Monthly_Emp_Update_Scratch\wa_esd_emp


In [5]:
current_dir = os.getcwd()

In [122]:
esd_sa_url = "https://fortress.wa.gov/esd/employmentdata/docs/economic-reports/wa-historical-employment-seasonally-adjusted.xlsx"
sa_filename = esd_sa_url[-49:-5]+"_"+time.strftime("%m%d%Y")+esd_sa_url[-5:]
def download_esd_sa():
    start = time.time()
    esdfile = urllib.URLopener()
    esdfile.retrieve(esd_sa_url, os.getcwd()+"/"+ sa_filename)
    end = time.time()
    processtime = end - start
    print "{0: .3}".format(processtime)
    

In [123]:
download_esd_sa()

 1.1


In [13]:
help (urllib)

Help on module urllib:

NAME
    urllib - Open an arbitrary URL.

FILE
    c:\users\jalal\anaconda2\lib\urllib.py

DESCRIPTION
    See the following document for more info on URLs:
    "Names and Addresses, URIs, URLs, URNs, URCs", at
    http://www.w3.org/pub/WWW/Addressing/Overview.html
    
    See also the HTTP spec (from which the error codes are derived):
    "HTTP - Hypertext Transfer Protocol", at
    http://www.w3.org/pub/WWW/Protocols/
    
    Related standards and specs:
    - RFC1808: the "relative URL" spec. (authoritative status)
    - RFC1738 - the "URL standard". (authoritative status)
    - RFC1630 - the "URI spec". (informational status)
    
    The object returned by URLopener().open(file) will differ per
    protocol.  All you know is that is has methods read(), readline(),
    readlines(), fileno(), close() and info().  The read*(), fileno()
    and close() methods work like those of open files.
    The info() method returns a mimetools.Message object which can b

In [124]:
# Check the number of worksheet in the .xlsx file & list the name of the worksheets
#included in the analysis

def check_worksheet():
    esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
    #get the list of worksheets from the excelfile
    esd_sa_ws_master = {sheet_name: esd_sa_file.parse(sheet_name) 
              for sheet_name in esd_sa_file.sheet_names}

    a_list = []
    count = 0
    count_total = 0
    for i in esd_sa_ws_master:
        if i == "Seattle MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Bremerton MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Tacoma MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        else:
            count_total = count_total+1


    print "List of worksheets to be included in the analysis: "

    count_ws = 0
    for x in sorted(a_list):
        count_ws = count_ws+1 
        print str(count_ws)+". "+ x

    print "Total "+str(count)+" "+"out of "+ str(count_total)+ " Worrksheets are in the PSRC region."

check_worksheet()

List of worksheets to be included in the analysis: 
1. Bremerton MSA
2. Seattle MSA
3. Tacoma MSA
Total 3 out of 12 Worrksheets are in the PSRC region.


In [94]:
## EXAMPLE: Read the worksheet into dataframe from a excel file:

#esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
#df_sa_kin_sno = esd_sa_file.parse("Seattle MSA", header =1).head()

In [246]:
#read the required worksheets into dataframes

df_sa_kin_sno = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])
df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
df_sa_pie = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Tacoma MSA", header = 1, index_col=[1])


In [183]:
type(df_sa_pie)

pandas.core.frame.DataFrame

In [182]:
df_sa_pie.shape

(315, 11)

In [185]:
df_sa_pie.dtypes

id
year                      int64
month                    object
Pierce: Total NonFarm     int64
Pierce: Consruction       int64
Pierce: FIRE              int64
Pierce: Manufacturing     int64
Pierce: Retail            int64
Pierce: Service           int64
Pierce: WTU               int64
Pierce: Government        int64
Pierce: Education         int64
dtype: object

In [188]:
df_sa_pie.columns.tolist()

['year',
 'month',
 'Pierce: Total NonFarm',
 'Pierce: Consruction',
 'Pierce: FIRE',
 'Pierce: Manufacturing',
 'Pierce: Retail',
 'Pierce: Service',
 'Pierce: WTU',
 'Pierce: Government',
 'Pierce: Education']

In [125]:
#read the required worksheets into dataframes

df_sa_kin_sno = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])

#transpose the dataframe

df_sa_kin_sno = df_sa_kin_sno.transpose()

#transposing caused the date column to be the index column, so
#extracting the date information to another column named "date_stamp"

df_sa_kin_sno['date_stamp'] = df_sa_kin_sno.index

#deleting the first data row (not the header) containing NAICS info 
#entirely for formatting purposes 

df_sa_kin_sno = df_sa_kin_sno.ix[1:]

#resetting the index of the dataframe
#as we already copied the date info into another column, nothing to worry

df_sa_kin_sno = df_sa_kin_sno.reset_index(drop =True)

#renaming the index colum from "NAICS Industy" to "id" 

df_sa_kin_sno.columns.names = ["id"]

#get a column for year and one for month (note the foramting for month)
df_sa_kin_sno["year"] = df_sa_kin_sno["date_stamp"].dt.year
df_sa_kin_sno["month"] = df_sa_kin_sno["date_stamp"].dt.strftime("%b")

#get the month column to string (not essential)
df_sa_kin_sno["month"] = df_sa_kin_sno["month"].astype("str")

#get the list of column names, any of the options below works just fine
# list(df_sa_kin_sno) or list(df_sa_kin_sno.columns.values) works too
kin_sno_list = df_sa_kin_sno.columns.values.tolist()


#define PSRC sectors by adding columns for each of the eight sectors and total nonfarm jobs:
df_sa_kin_sno["King & Snohomish: Total NonFarm"] = df_sa_kin_sno["Total Nonfarm"] 
df_sa_kin_sno["King & Snohomish: Consruction"] = df_sa_kin_sno[kin_sno_list[3]] + df_sa_kin_sno[kin_sno_list[4]]
df_sa_kin_sno["King & Snohomish: FIRE"] = df_sa_kin_sno[kin_sno_list[33]]
df_sa_kin_sno["King & Snohomish: Manufacturing"] = df_sa_kin_sno[kin_sno_list[8]]
df_sa_kin_sno["King & Snohomish: Retail"] = df_sa_kin_sno[kin_sno_list[20]]
df_sa_kin_sno["King & Snohomish: Service"] = df_sa_kin_sno[kin_sno_list[17]] - df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[33]] - df_sa_kin_sno[kin_sno_list[62]]                                                     
df_sa_kin_sno["King & Snohomish: WTU"] = df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[20]]
df_sa_kin_sno["King & Snohomish: Government"] = df_sa_kin_sno[kin_sno_list[62]] - df_sa_kin_sno[kin_sno_list[65]] - df_sa_kin_sno[kin_sno_list[67]]
df_sa_kin_sno["King & Snohomish: Education"] = df_sa_kin_sno[kin_sno_list[65]] + df_sa_kin_sno[kin_sno_list[67]]

#delete columns that we don't need anymore (all except the year, month and columns created for sectors)
#df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[range(69)], axis = 1) works for this operation too!
df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[:69], axis = 1)

#ATTENTION: """just for this iteration, won't need for future runs""", deleting the row that contains Apr16 info for Seattle MSA
df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.index[[315]])

#putting in a check-point in place to make sure the sector columns add up to the total jobs:
#df_sa_kit["Kitsap: Total NonFarm"].isin(df_sa_kit[kit_col_list[3:]].sum(axis=1)).all() == True.... works too for this step.

if df_sa_kin_sno["King & Snohomish: Total NonFarm"].isin(df_sa_kin_sno.ix[:,3:].sum(axis=1)).all() == True:
    print "King & Snohomish Counties: All industry sectors adds up to total NonFarm jobs..."
else:
    print "ERROR!!"


df_sa_kin_sno.tail()

King & Snohomish Counties: All industry sectors adds up to total NonFarm jobs...


id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,King & Snohomish: Education
310,2015,Nov,1608600,87900,83100,169800,169400,757400,126100,110400,104500
311,2015,Dec,1614000,89400,82900,170100,169700,759800,127200,110600,104300
312,2016,Jan,1619100,89800,84300,168900,173500,761900,127700,110400,102600
313,2016,Feb,1627000,91700,84000,167800,174300,766700,127500,110900,104100
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,104300


In [126]:
df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
df_sa_kit = df_sa_kit.transpose()
df_sa_kit['date_stamp'] = df_sa_kit.index
df_sa_kit = df_sa_kit.ix[1:]
df_sa_kit = df_sa_kit.reset_index(drop =True)
df_sa_kit.columns.names = ["id"]
df_sa_kit["year"] = df_sa_kit["date_stamp"].dt.year
df_sa_kit["month"] = df_sa_kit["date_stamp"].dt.strftime("%b")

#get the month column to string (not essential)
df_sa_kit["month"] = df_sa_kit["month"].astype("str")

kit_list = df_sa_kit.columns.values.tolist()

df_sa_kit["Kitsap: Total NonFarm"] = df_sa_kit["Total Nonfarm"] 
df_sa_kit["Kitsap: Consruction"] = df_sa_kit[kit_list[3]]
#add new column for FIRE with zero as values with index aligned with the dataframe
df_sa_kit["Kitsap: FIRE"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)
df_sa_kit["Kitsap: Manufacturing"] = df_sa_kit[kit_list[4]]
df_sa_kit["Kitsap: Retail"] = df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Service"] = df_sa_kit[kit_list[5]] - df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: WTU"] = df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Government"] = df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: Education"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)

df_sa_kit = df_sa_kit.drop(df_sa_kit.columns[:16], axis = 1)

if df_sa_kit["Kitsap: Total NonFarm"].isin(df_sa_kit.ix[:,3:].sum(axis=1)).all() == True:
    print "Kitsap County: All industry sectors adds up to total NonFarm jobs..."
else:
    print "ERROR!!"


df_sa_kit.tail()

Kitsap County: All industry sectors adds up to total NonFarm jobs...


id,year,month,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
310,2015,Nov,88200,4600,0,2400,11000,37400,1600,31200,0
311,2015,Dec,88400,4400,0,2600,11100,35000,3600,31700,0
312,2016,Jan,88300,4200,0,2700,10900,37000,1600,31900,0
313,2016,Feb,88300,4300,0,2700,11000,36600,1700,32000,0
314,2016,Mar,88400,4400,0,2600,10800,35300,3100,32200,0


In [127]:
df_sa_pie = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Tacoma MSA", header = 1, index_col=[1])
df_sa_pie = df_sa_pie.transpose()
df_sa_pie['date_stamp'] = df_sa_pie.index
df_sa_pie = df_sa_pie.ix[1:]
df_sa_pie = df_sa_pie.reset_index(drop =True)
df_sa_pie.columns.names = ["id"]
df_sa_pie["year"] = df_sa_pie["date_stamp"].dt.year
df_sa_pie["month"] = df_sa_pie["date_stamp"].dt.strftime("%b")

#get the month column to string (not essential)
df_sa_pie["month"] = df_sa_pie["month"].astype("str")

pie_list = df_sa_pie.columns.values.tolist()

df_sa_pie["Pierce: Total NonFarm"] = df_sa_pie["Total Nonfarm"] 
df_sa_pie["Pierce: Consruction"] = df_sa_pie[pie_list[3]] + df_sa_pie[pie_list[4]]
df_sa_pie["Pierce: FIRE"] = df_sa_pie[pie_list[15]]
df_sa_pie["Pierce: Manufacturing"] = df_sa_pie[pie_list[6]]
df_sa_pie["Pierce: Retail"] = df_sa_pie[pie_list[10]]
df_sa_pie["Pierce: Service"] = df_sa_pie[pie_list[7]] - df_sa_pie[pie_list[8]] - df_sa_pie[pie_list[15]] - df_sa_pie[pie_list[25]]                                                     
df_sa_pie["Pierce: WTU"] = df_sa_pie[pie_list[8]] - df_sa_pie[pie_list[10]]
df_sa_pie["Pierce: Government"] = df_sa_pie[pie_list[25]] - df_sa_pie[pie_list[28]] - df_sa_pie[pie_list[30]]
df_sa_pie["Pierce: Education"] = df_sa_pie[pie_list[28]] + df_sa_pie[pie_list[30]]

df_sa_pie = df_sa_pie.drop(df_sa_pie.columns[:32], axis = 1)    

if df_sa_pie["Pierce: Total NonFarm"].isin(df_sa_pie.ix[:,3:].sum(axis=1)).all() == True:
    print "Pierce County: All industry sectors adds up to total NonFarm jobs..."
else:
    print "ERROR!!"


df_sa_pie.tail()

Pierce County: All industry sectors adds up to total NonFarm jobs...


id,year,month,Pierce: Total NonFarm,Pierce: Consruction,Pierce: FIRE,Pierce: Manufacturing,Pierce: Retail,Pierce: Service,Pierce: WTU,Pierce: Government,Pierce: Education
310,2015,Nov,306000,21200,13000,17500,39900,129400,26800,34900,23300
311,2015,Dec,307300,20900,14000,17300,40100,129800,26700,35500,23000
312,2016,Jan,308100,20800,13600,17200,39600,131900,26300,35400,23300
313,2016,Feb,310500,21100,13600,17400,39400,133300,26600,35400,23700
314,2016,Mar,310900,21500,13700,17500,39600,131900,27000,35700,24000


In [128]:
df_sa_psrc = pd.merge(pd.merge(df_sa_kin_sno, df_sa_pie, on = ["year", "month"]), df_sa_kit, on = ["year", "month"])

In [134]:
df_sa_psrc.tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Pierce: Education,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
310,2015,Nov,1608600,87900,83100,169800,169400,757400,126100,110400,...,23300,88200,4600,0,2400,11000,37400,1600,31200,0
311,2015,Dec,1614000,89400,82900,170100,169700,759800,127200,110600,...,23000,88400,4400,0,2600,11100,35000,3600,31700,0
312,2016,Jan,1619100,89800,84300,168900,173500,761900,127700,110400,...,23300,88300,4200,0,2700,10900,37000,1600,31900,0
313,2016,Feb,1627000,91700,84000,167800,174300,766700,127500,110900,...,23700,88300,4300,0,2700,11000,36600,1700,32000,0
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,...,24000,88400,4400,0,2600,10800,35300,3100,32200,0


In [168]:
psrc_list = df_sa_psrc.columns.values.tolist()

In [169]:
psrc_list

['year',
 'month',
 'King & Snohomish: Total NonFarm',
 'King & Snohomish: Consruction',
 'King & Snohomish: FIRE',
 'King & Snohomish: Manufacturing',
 'King & Snohomish: Retail',
 'King & Snohomish: Service',
 'King & Snohomish: WTU',
 'King & Snohomish: Government',
 'King & Snohomish: Education',
 'Pierce: Total NonFarm',
 'Pierce: Consruction',
 'Pierce: FIRE',
 'Pierce: Manufacturing',
 'Pierce: Retail',
 'Pierce: Service',
 'Pierce: WTU',
 'Pierce: Government',
 'Pierce: Education',
 'Kitsap: Total NonFarm',
 'Kitsap: Consruction',
 'Kitsap: FIRE',
 'Kitsap: Manufacturing',
 'Kitsap: Retail',
 'Kitsap: Service',
 'Kitsap: WTU',
 'Kitsap: Government',
 'Kitsap: Education']

In [189]:
psrc_list[02]

'King & Snohomish: Total NonFarm'

In [190]:
df_sa_psrc["Region: Construction"] = df_sa_psrc[psrc_list[3]] + df_sa_psrc[psrc_list[12]] + df_sa_psrc[psrc_list[21]]
df_sa_psrc["Region: FIRE"] = df_sa_psrc[psrc_list[4]] + df_sa_psrc[psrc_list[13]] + df_sa_psrc[psrc_list[22]]
df_sa_psrc["Region: Manufacturing"] = df_sa_psrc[psrc_list[5]] + df_sa_psrc[psrc_list[14]] + df_sa_psrc[psrc_list[23]]
df_sa_psrc["Region: Retail"] = df_sa_psrc[psrc_list[6]] + df_sa_psrc[psrc_list[15]] + df_sa_psrc[psrc_list[24]]
df_sa_psrc["Region: Service"] = df_sa_psrc[psrc_list[7]] + df_sa_psrc[psrc_list[16]] + df_sa_psrc[psrc_list[25]]
df_sa_psrc["Region: WTU"] = df_sa_psrc[psrc_list[8]] + df_sa_psrc[psrc_list[17]] + df_sa_psrc[psrc_list[26]]
df_sa_psrc["Region: Education"] = df_sa_psrc[psrc_list[10]] + df_sa_psrc[psrc_list[19]] + df_sa_psrc[psrc_list[28]]
df_sa_psrc["Region: Government"] = df_sa_psrc[psrc_list[9]] + df_sa_psrc[psrc_list[18]] + df_sa_psrc[psrc_list[27]]
df_sa_psrc["Region: Total NonFarm"] = df_sa_psrc[psrc_list[2]] + df_sa_psrc[psrc_list[11]] + df_sa_psrc[psrc_list[20]]

In [89]:
df_sa_psrc.dtypes

id
year                                int64
month                              object
King & Snohomish: Total NonFarm     int64
King & Snohomish: Consruction       int64
King & Snohomish: FIRE              int64
King & Snohomish: Manufacturing     int64
King & Snohomish: Retail            int64
King & Snohomish: Service           int64
King & Snohomish: WTU               int64
King & Snohomish: Government        int64
King & Snohomish: Education         int64
Pierce: Total NonFarm               int64
Pierce: Consruction                 int64
Pierce: FIRE                        int64
Pierce: Manufacturing               int64
Pierce: Retail                      int64
Pierce: Service                     int64
Pierce: WTU                         int64
Pierce: Government                  int64
Pierce: Education                   int64
Kitsap: Total NonFarm               int64
Kitsap: Consruction                 int64
Kitsap: FIRE                        int64
Kitsap: Manufacturing          

In [135]:
df_sa_psrc.shape

(315, 29)

In [193]:
df_sa_region = df_sa_psrc[[0,1, 2, 11,20,-1]][df_sa_psrc.month== active_month]

In [194]:
df_sa_region.tail()

id,year,month,King & Snohomish: Total NonFarm,Pierce: Total NonFarm,Kitsap: Total NonFarm,Region: Total NonFarm
266,2012,Mar,1446800,270200,83600,1800600
278,2013,Mar,1490000,274600,83300,1847900
290,2014,Mar,1533400,282700,85000,1901100
302,2015,Mar,1578600,291700,87500,1957800
314,2016,Mar,1629200,310900,88400,2028500


In [195]:
df_sa_psrc[df_sa_psrc.month== "Mar"].tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Kitsap: Education,Region: Construction,Region: FIRE,Region: Manufacturing,Region: Retail,Region: Service,Region: WTU,Region: Education,Region: Government,Region: Total NonFarm
266,2012,Mar,1446800,64200,77500,164700,143000,680600,114200,107700,...,0,83400,88900,183100,184900,830400,140600,117700,171600,1800600
278,2013,Mar,1490000,71100,80000,170500,150300,698800,115300,107200,...,0,91400,91700,189700,192100,849400,143400,119300,170900,1847900
290,2014,Mar,1533400,76500,80500,169200,159300,722200,118800,107900,...,0,98700,92900,188900,202300,878500,147400,121800,170600,1901100
302,2015,Mar,1578600,86400,81200,171100,167400,738800,122700,108800,...,0,109900,93700,190800,213200,898200,152000,125400,174600,1957800
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,...,0,118400,98000,187700,224200,932600,159400,128300,179900,2028500


In [108]:
(df_sa_psrc[(df_sa_psrc.month == "Mar") & (df_sa_psrc.year == 2000)].tail() + df_sa_psrc[(df_sa_psrc.month == "Apr") & (df_sa_psrc.year == 2000)].tail())

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Pierce: Education,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df_sa_psrc.query('month== Mar').tail()

UndefinedVariableError: name 'Mar' is not defined

In [30]:
#delete unnecessary worksheets from the region

def delete_ws():
    wb_sa = load_workbook(os.getcwd()+"/"+ sa_filename)

    del_sheets = []

    for i in wb_sa.get_sheet_names():
        if i == "Seattle MSA":
            pass
        elif i == "Tacoma MSA":
            pass
        elif i == "Bremerton MSA":
            pass
        elif i == "Washington State":
            pass
        else:
            del_sheets.append(i)
    #print del_sheets
#now delete the bad sheets
    [wb_sa.remove_sheet(wb_sa.get_sheet_by_name(sheet)) for sheet in del_sheets]
#if you get a ValueError, this means you are incorrectly requesting a sheet

    wb_sa.save(os.getcwd()+"/"+ sa_filename)    

delete_ws()
    
    

[]


In [284]:
sa_book = load_workbook(os.getcwd()+"/"+ sa_filename)
sa_writer = pd.ExcelWriter(os.getcwd()+"/"+ sa_filename, engine='openpyxl') 
sa_writer.book = sa_book
sa_writer.sheets = dict((ws.title, ws) for ws in sa_book.worksheets)
df_sa_psrc.to_excel(sa_writer, "RegionMaster_JJ", index =False)
sa_writer.save()
